In [2]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from datetime import datetime
from  backtest import Backtester,  get_ohlc_history, create_price_fig, evaluate_backtest

In [3]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [4]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's').dt.tz_localize('UTC')
    return df

In [5]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,6,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
#df.set_index("time", inplace=True)
#df = df.between_time("13:00", "16:00")
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-06-02 00:00:00+00:00,1.13463,1.13463,1.13398,1.13398,2,70,0
1,2025-06-02 00:01:00+00:00,1.13398,1.13398,1.13398,1.13398,1,200,0
2,2025-06-02 00:02:00+00:00,1.13398,1.13398,1.13398,1.13398,1,200,0
3,2025-06-02 00:03:00+00:00,1.13398,1.13398,1.13390,1.13390,2,200,0
4,2025-06-02 00:04:00+00:00,1.13390,1.13390,1.13382,1.13382,2,200,0
...,...,...,...,...,...,...,...,...
68993,2025-08-06 23:00:00+00:00,1.16566,1.16587,1.16564,1.16577,49,0,0
68994,2025-08-06 23:01:00+00:00,1.16576,1.16586,1.16569,1.16586,45,0,0
68995,2025-08-06 23:02:00+00:00,1.16586,1.16592,1.16586,1.16591,38,0,0
68996,2025-08-06 23:03:00+00:00,1.16591,1.16591,1.16567,1.16568,34,0,0


In [6]:
# === Calculate indicators ===
df["atr"] = df["high"].rolling(14).max() - df["low"].rolling(14).min()
df["vwap"] = (df["close"] * df["tick_volume"]).cumsum() / df["tick_volume"].cumsum()
df["volume_spike"] = df["tick_volume"] > df["tick_volume"].rolling(10).mean() * 1.5

# === Signal Generation ===
def generate_signals(df):
    signals = []
    for i in range(14, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i-1]

        cond1 = row["close"] < row["vwap"] - 2 * row["atr"]
        cond2 = row["spread"] < 0.0002
        cond3 = row["volume_spike"]
        cond4 = row["close"] > row["open"] and prev["close"] < prev["open"]

        if cond1 and cond2 and cond3 and cond4: 
            signals.append(i)
    return signals

signals = generate_signals(df)

# === Backtest ===
results = []
rr = 3  # risk-reward ratio

for i in signals:
    entry_price = df.iloc[i]["close"]
    sl = entry_price - 0.5 * df.iloc[i]["atr"]
    tp = entry_price + rr * (entry_price - sl)

    for j in range(i+1, min(i+60, len(df))):  # look ahead for next ~2 hours
        high = df.iloc[j]["high"]
        low = df.iloc[j]["low"]
        if low <= sl:
            results.append(("loss", entry_price, sl))
            break
        elif high >= tp:
            results.append(("win", entry_price, tp))
            break
    else:
        # time-based exit
        close_exit = df.iloc[i+60]["close"]
        if close_exit > entry_price:
            results.append(("win", entry_price, close_exit))
        else:
            results.append(("loss", entry_price, close_exit))
# === Metrics ===
wins = [r for r in results if r[0] == "win"]
losses = [r for r in results if r[0] == "loss"]
win_rate = len(wins) / len(results)
r_r_ratio = np.mean([abs(w[2]-w[1]) for w in wins]) / np.mean([abs(l[2]-l[1]) for l in losses])


entry_price = df.iloc[i]["close"]
df['sl'] = entry_price - 0.5 * df.iloc[i]["atr"]
df['tp'] = entry_price + rr * (entry_price - sl)

print(f"Total trades: {len(results)}")
print(f"Win rate: {win_rate:.2%}")
print(f"Risk-Reward Ratio: {r_r_ratio:.2f}")
print(results)


Total trades: 124
Win rate: 33.06%
Risk-Reward Ratio: 2.43
[('win', np.float64(1.13805), np.float64(1.13831)), ('loss', np.float64(1.13723), np.float64(1.13709)), ('loss', np.float64(1.13724), np.float64(1.13697)), ('win', np.float64(1.13713), np.float64(1.1376999999999997)), ('win', np.float64(1.13756), np.float64(1.1377999999999995)), ('loss', np.float64(1.13769), np.float64(1.137555)), ('loss', np.float64(1.13769), np.float64(1.1375450000000003)), ('win', np.float64(1.13784), np.float64(1.1383949999999996)), ('loss', np.float64(1.13837), np.float64(1.138175)), ('win', np.float64(1.13752), np.float64(1.13843)), ('loss', np.float64(1.13821), np.float64(1.13789)), ('loss', np.float64(1.13776), np.float64(1.137385)), ('loss', np.float64(1.13791), np.float64(1.1377549999999998)), ('loss', np.float64(1.13729), np.float64(1.13701)), ('loss', np.float64(1.13676), np.float64(1.13659)), ('win', np.float64(1.13587), np.float64(1.13743)), ('loss', np.float64(1.13644), np.float64(1.1359849999999

In [7]:
prev = df.iloc[i-1]
def entring_stoploss_profit(row) :
    idx = row.name
    if idx == 0:
        return None
    prev = df.iloc[idx - 1]

    if (
        row["close"] < row["vwap"] - 2 * row["atr"]
        and row["spread"] < 0.0002
        and row["volume_spike"]
        and row["close"] > row["open"]
        and prev["close"] < prev["open"]
    ):
        return "buy"
    return None
df["signal"] = df.apply(entring_stoploss_profit, axis=1)
df.copy().dropna()

,time,open,high,low,close,tick_volume,spread,real_volume,atr,vwap,volume_spike,sl,tp,signal
2373,2025-06-03 15:34:00+00:00,1.13783,1.13825,1.13772,1.13805,95,0,0,0.00105,1.141186,True,1.157165,1.158225,buy
2747,2025-06-03 21:48:00+00:00,1.13713,1.13723,1.13707,1.13723,38,0,0,0.00064,1.140699,True,1.157165,1.158225,buy
2780,2025-06-03 22:21:00+00:00,1.13714,1.13730,1.13713,1.13724,34,0,0,0.00054,1.140679,True,1.157165,1.158225,buy
2965,2025-06-04 01:27:00+00:00,1.13712,1.13718,1.13710,1.13713,27,0,0,0.00038,1.140607,True,1.157165,1.158225,buy
3007,2025-06-04 02:09:00+00:00,1.13745,1.13756,1.13745,1.13756,18,0,0,0.00016,1.140593,True,1.157165,1.158225,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68154,2025-08-06 09:01:00+00:00,1.15770,1.15812,1.15770,1.15810,60,0,0,0.00047,1.159576,True,1.157165,1.158225,buy
68159,2025-08-06 09:06:00+00:00,1.15803,1.15827,1.15802,1.15819,79,0,0,0.00062,1.159576,True,1.157165,1.158225,buy
68214,2025-08-06 10:01:00+00:00,1.15771,1.15777,1.15758,1.15773,98,0,0,0.00072,1.159574,True,1.157165,1.158225,buy
68259,2025-08-06 10:46:00+00:00,1.15757,1.15800,1.15757,1.15786,73,0,0,0.00074,1.159573,True,1.157165,1.158225,buy


In [8]:

# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' :#and not num_open_trades:
        volume = 400000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        volume = 400000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["low"] <= data["sl"] or data["high"] >= data["tp"]: 
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell'  :
            orders.close_trade(trade)

In [9]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,400000,2025-06-03 15:34:00+00:00,1.13783,2025-06-03 15:35:00+00:00,1.13805,0,0,{},88.0,-28.0,60.0,60.0,10060.0
1,closed,EURUSD,buy,400000,2025-06-03 21:48:00+00:00,1.13713,2025-06-03 21:49:00+00:00,1.13723,0,0,{},40.0,-28.0,12.0,72.0,10072.0
2,closed,EURUSD,buy,400000,2025-06-03 22:21:00+00:00,1.13714,2025-06-03 22:22:00+00:00,1.13724,0,0,{},40.0,-28.0,12.0,84.0,10084.0
3,closed,EURUSD,buy,400000,2025-06-04 01:27:00+00:00,1.13712,2025-06-04 01:28:00+00:00,1.13713,0,0,{},4.0,-28.0,-24.0,60.0,10060.0
4,closed,EURUSD,buy,400000,2025-06-04 02:09:00+00:00,1.13745,2025-06-04 02:10:00+00:00,1.13756,0,0,{},44.0,-28.0,16.0,76.0,10076.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,closed,EURUSD,buy,400000,2025-08-06 09:01:00+00:00,1.1577,2025-08-06 09:06:00+00:00,1.15803,0,0,{},132.0,-28.0,104.0,2672.0,12672.0
120,closed,EURUSD,buy,400000,2025-08-06 09:06:00+00:00,1.15803,2025-08-06 09:07:00+00:00,1.15819,0,0,{},64.0,-28.0,36.0,2708.0,12708.0
121,closed,EURUSD,buy,400000,2025-08-06 10:01:00+00:00,1.15771,2025-08-06 10:15:00+00:00,1.1576,0,0,{},-44.0,-28.0,-72.0,2636.0,12636.0
122,closed,EURUSD,buy,400000,2025-08-06 10:46:00+00:00,1.15757,2025-08-06 11:07:00+00:00,1.15817,0,0,{},240.0,-28.0,212.0,2848.0,12848.0


In [10]:
mm = bt.plot_balance()
display(mm)

In [11]:
evaluate_backtest(bt.trades)

biggest_profit: 492.0
daily_drawdown: -200.0
max_drawown: -268.0
avg_win: 59.79
avg_loss: -78.0
count_profit_trades: 115
count_loss_trades: 8
Win Rate1: 93.50%
rrr: 0.77


,order_type,profit
0,buy,6252.0


c:\Users\hp\python_finance_market_projects\backtest.py:368: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown
0,closed,EURUSD,buy,400000,2025-06-03 15:34:00+00:00,1.13783,2025-06-03 15:35:00+00:00,1.13805,0,0,{},88.0,-28.0,60.0,60.0,10060.0,60.0,0.0
1,closed,EURUSD,buy,400000,2025-06-03 21:48:00+00:00,1.13713,2025-06-03 21:49:00+00:00,1.13723,0,0,{},40.0,-28.0,12.0,72.0,10072.0,72.0,0.0
2,closed,EURUSD,buy,400000,2025-06-03 22:21:00+00:00,1.13714,2025-06-03 22:22:00+00:00,1.13724,0,0,{},40.0,-28.0,12.0,84.0,10084.0,84.0,0.0
3,closed,EURUSD,buy,400000,2025-06-04 01:27:00+00:00,1.13712,2025-06-04 01:28:00+00:00,1.13713,0,0,{},4.0,-28.0,-24.0,60.0,10060.0,84.0,-24.0
4,closed,EURUSD,buy,400000,2025-06-04 02:09:00+00:00,1.13745,2025-06-04 02:10:00+00:00,1.13756,0,0,{},44.0,-28.0,16.0,76.0,10076.0,84.0,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,closed,EURUSD,buy,400000,2025-08-06 03:01:00+00:00,1.15783,2025-08-06 03:04:00+00:00,1.15828,0,0,{},180.0,-28.0,152.0,2704.0,12704.0,2704.0,0.0
118,closed,EURUSD,buy,400000,2025-08-06 08:01:00+00:00,1.1581,2025-08-06 08:02:00+00:00,1.15824,0,0,{},56.0,-28.0,28.0,2568.0,12568.0,2704.0,-136.0
119,closed,EURUSD,buy,400000,2025-08-06 09:01:00+00:00,1.1577,2025-08-06 09:06:00+00:00,1.15803,0,0,{},132.0,-28.0,104.0,2672.0,12672.0,2704.0,-32.0
120,closed,EURUSD,buy,400000,2025-08-06 09:06:00+00:00,1.15803,2025-08-06 09:07:00+00:00,1.15819,0,0,{},64.0,-28.0,36.0,2708.0,12708.0,2708.0,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown
31,closed,EURUSD,buy,400000,2025-07-17 14:46:00+00:00,1.15755,2025-07-17 14:50:00+00:00,1.15725,0,0,{},-120.0,-28.0,-148.0,140.0,10140.0,328.0,-188.0
39,closed,EURUSD,buy,400000,2025-07-29 03:52:00+00:00,1.15887,2025-07-29 03:53:00+00:00,1.15883,0,0,{},-16.0,-28.0,-44.0,404.0,10404.0,448.0,-44.0
46,closed,EURUSD,buy,400000,2025-07-29 13:51:00+00:00,1.15763,2025-07-29 14:03:00+00:00,1.15713,0,0,{},-200.0,-28.0,-228.0,288.0,10288.0,516.0,-228.0
113,closed,EURUSD,buy,400000,2025-08-05 22:58:00+00:00,1.15729,2025-08-05 23:33:00+00:00,1.15717,0,0,{},-48.0,-28.0,-76.0,2340.0,12340.0,2440.0,-100.0
114,closed,EURUSD,buy,400000,2025-08-06 01:00:00+00:00,1.15732,2025-08-06 01:32:00+00:00,1.15727,0,0,{},-20.0,-28.0,-48.0,2292.0,12292.0,2440.0,-148.0
117,closed,EURUSD,buy,400000,2025-08-06 03:21:00+00:00,1.1577,2025-08-06 04:00:00+00:00,1.15736,0,0,{},-136.0,-28.0,-164.0,2540.0,12540.0,2704.0,-164.0
121,closed,EURUSD,buy,400000,2025-08-06 10:01:00+00:00,1.15771,2025-08-06 10:15:00+00:00,1.1576,0,0,{},-44.0,-28.0,-72.0,2636.0,12636.0,2708.0,-72.0
123,closed,EURUSD,buy,400000,2025-08-06 11:32:00+00:00,1.15735,2025-08-06 11:34:00+00:00,1.15725,0,0,{},-40.0,-28.0,-68.0,2780.0,12780.0,2848.0,-68.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,buy,400000,2025-06-03 15:34:00+00:00,1.13783,2025-06-03 15:35:00+00:00,1.13805,0,0,...,-28.0,60.0,60.0,10060.0,60.0,0.0,1,15,2025-06,2025
1,closed,EURUSD,buy,400000,2025-06-03 21:48:00+00:00,1.13713,2025-06-03 21:49:00+00:00,1.13723,0,0,...,-28.0,12.0,72.0,10072.0,72.0,0.0,1,21,2025-06,2025
2,closed,EURUSD,buy,400000,2025-06-03 22:21:00+00:00,1.13714,2025-06-03 22:22:00+00:00,1.13724,0,0,...,-28.0,12.0,84.0,10084.0,84.0,0.0,1,22,2025-06,2025
3,closed,EURUSD,buy,400000,2025-06-04 01:27:00+00:00,1.13712,2025-06-04 01:28:00+00:00,1.13713,0,0,...,-28.0,-24.0,60.0,10060.0,84.0,-24.0,2,1,2025-06,2025
4,closed,EURUSD,buy,400000,2025-06-04 02:09:00+00:00,1.13745,2025-06-04 02:10:00+00:00,1.13756,0,0,...,-28.0,16.0,76.0,10076.0,84.0,-8.0,2,2,2025-06,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,closed,EURUSD,buy,400000,2025-08-06 09:01:00+00:00,1.1577,2025-08-06 09:06:00+00:00,1.15803,0,0,...,-28.0,104.0,2672.0,12672.0,2704.0,-32.0,2,9,2025-08,2025
120,closed,EURUSD,buy,400000,2025-08-06 09:06:00+00:00,1.15803,2025-08-06 09:07:00+00:00,1.15819,0,0,...,-28.0,36.0,2708.0,12708.0,2708.0,0.0,2,9,2025-08,2025
121,closed,EURUSD,buy,400000,2025-08-06 10:01:00+00:00,1.15771,2025-08-06 10:15:00+00:00,1.1576,0,0,...,-28.0,-72.0,2636.0,12636.0,2708.0,-72.0,2,10,2025-08,2025
122,closed,EURUSD,buy,400000,2025-08-06 10:46:00+00:00,1.15757,2025-08-06 11:07:00+00:00,1.15817,0,0,...,-28.0,212.0,2848.0,12848.0,2848.0,0.0,2,10,2025-08,2025


max_drawdown -268.0
